In [7]:
import pandas as pd
import multiprocessing
import tqdm
import pickle
import re

from mypostgres import PostgresConnection

from scraper import carProperties

from joblib import Parallel, delayed
num_cores = multiprocessing.cpu_count()

import getpass


## Scrape Pages in parallel

In [2]:
cars_list = []

def get_car_data(url):
    return carProperties(url).get_data()

with open('/home/leon/Documents/repos/car-prices-analysis/crawler/links copy.csv','r') as f:
    
    urls = [l.strip('\n') for l in f.readlines()]
    cars_list = Parallel(n_jobs=num_cores)(delayed(get_car_data)(url) for url in tqdm.tqdm(urls))

    cars_df = pd.concat(cars_list,axis=1).T

100%|██████████| 760/760 [05:06<00:00,  2.48it/s]


In [23]:
cars_df = cars_df.infer_objects()
cars_df['power'] = cars_df['power'].astype('int')
cars_df[['brand','model']] = cars_df[['brand','model']].astype('category')

In [39]:
with open('cars_data.pkl','wb') as f:
    pickle.dump(cars_df,f)
    

In [111]:
with pgs.PostgresConnection(dbname='random',host='192.168.1.218',user='leon',schema='cars',password=getpass.getpass('PSQL password: ')) as conn:
    conn.create_table_from_df(cars_df,'fct_cars')
    conn.insert_df(cars_df,'fct_cars')

In [ ]:
cars_df.describe()

todo:
- plot correlation heatmap for exogenous variables
- remove multicolinearity
- check for other biases